# 개요

# Prepare

# 1. 초기 세팅

## Library & data load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
# 처리용 df와 오리지널 버전 남겨놓음
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/prd_auction/data_origin/auction_202206.csv",encoding='euc_kr')
org_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/prd_auction/data_origin/auction_202206.csv",encoding='euc_kr')

# Pre-Processing

## Basic info

In [ ]:
df.head()

,품목명,단위,등급,가격,산지,친환경구분,입력일
0,[가자미수입]가자미수입,4.5kg,보통(3등),18500,수입 러시아\t,일반,20220601
1,[가자미수입]가자미수입,4.5kg,보통(3등),16000,수입 러시아\t,일반,20220601
2,[가자미수입]가자미수입,4.5kg,보통(3등),18500,수입 러시아\t,일반,20220601
3,[가자미수입]가자미수입,4.5kg,보통(3등),18500,수입 러시아\t,일반,20220601
4,[가지]가지,5kg,특(1등),11000,경남 진주시,일반,20220601


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1072668 entries, 0 to 1072667
Data columns (total 7 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   품목명     1072668 non-null  object
 1   단위      1072668 non-null  object
 2   등급      1072668 non-null  object
 3   가격      1072668 non-null  int64 
 4   산지      1071216 non-null  object
 5   친환경구분   1072668 non-null  object
 6   입력일     1072668 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 57.3+ MB


In [ ]:
# 산지 제외하고는 결측치 없는 것으로 확인
df.isnull().sum()

품목명         0
단위          0
등급          0
가격          0
산지       1452
친환경구분       0
입력일         0
dtype: int64

- 결측치 처리

In [ ]:
###
# 100만개 중 1450개 가량이므로 결측치 제거
df = df.dropna(axis=0)

- 영문명으로 전환

In [ ]:
df = df.rename(columns = {'품목명':'prd',
                          '단위':'scale','등급':'class',
                          '가격':'price',
                          '산지':'origin',
                          '친환경구분':'eco',
                          '입력일':'reg_date'})

## column 별 정보

### class

In [ ]:
# 이름 모두 영어로 변경
# 9등과 없음 모두 9로 변경
df['class'].value_counts().index

Index(['특(1등)', '상(2등)', '보통(3등)', '9등(등외)', '없음', '5등', '4등', '6등', '7등',
       '8등'],
      dtype='object')

In [ ]:
df111 = pd.DataFrame([['특(1등)', 1], ['상(2등)',2], ['보통(3등)',3], ['9등(등외)',9], ['없음',9], ['5등',5], ['4등',4], ['6등',6], ['7등',7],['8등',8]],columns=['class','new_class'])

In [ ]:
df2 = df.merge(df111)
df2 = df2.drop(columns='class')

In [ ]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,가자미,4.5,18500,수입 러시아\t,일반,20220601,3
1,가자미,4.5,16000,수입 러시아\t,일반,20220601,3
2,가자미,4.5,18500,수입 러시아\t,일반,20220601,3
3,가자미,4.5,18500,수입 러시아\t,일반,20220601,3
4,가자미,4.5,14000,수입 러시아\t,일반,20220603,3
...,...,...,...,...,...,...,...
1070137,토마토,10.0,4000,인천 강화군,일반,20220629,7
1070138,토마토,10.0,8000,인천 강화군,일반,20220630,7
1070139,토마토,5.0,4000,인천 강화군,일반,20220630,7
1070140,토마토,10.0,7500,강원 춘천시,일반,20220630,7


### eco

In [ ]:
# 영문명으로 전환
# 전환기 -> 유기농 재배로 전환중
df2.loc[df2["eco"]=="일반","eco"] = "normal"
df2.loc[df2["eco"]=="우수농산물","eco"] = "good_prd"
df2.loc[df2["eco"]=="무농약","eco"] = "ecofriendly"
df2.loc[df2["eco"]=="유기농","eco"] = "organic"
df2.loc[df2["eco"]=="품질인증","eco"] = "certified"
df2.loc[df2["eco"]=="전환기","eco"] = "ing"

In [ ]:
df2

,prd,scale,price,eco,reg_date,new_class,price_kg,state,city
0,가자미,4.50000,18500,normal,20220601,3,4111.11000,수입,러시아
1,가자미,4.50000,16000,normal,20220601,3,3555.56000,수입,러시아
2,가자미,4.50000,18500,normal,20220601,3,4111.11000,수입,러시아
3,가자미,4.50000,18500,normal,20220601,3,4111.11000,수입,러시아
4,가자미,4.50000,14000,normal,20220603,3,3111.11000,수입,러시아
...,...,...,...,...,...,...,...,...,...
1070137,토마토,10.00000,4000,normal,20220629,7,400.00000,인천,강화
1070138,토마토,10.00000,8000,normal,20220630,7,800.00000,인천,강화
1070139,토마토,5.00000,4000,normal,20220630,7,800.00000,인천,강화
1070140,토마토,10.00000,7500,normal,20220630,7,750.00000,강원,춘천


### reg_date

In [ ]:
df2['reg_date'] = df2['reg_date'].astype('str')

In [ ]:
df2['reg_date'] = pd.to_datetime(df2['reg_date'].astype('str'),format='%Y%m%d')

In [ ]:
df2

,prd,scale,price,eco,reg_date,new_class,price_kg,state,city
0,가자미,4.50000,18500,normal,2022-06-01,3,4111.11000,수입,러시아
1,가자미,4.50000,16000,normal,2022-06-01,3,3555.56000,수입,러시아
2,가자미,4.50000,18500,normal,2022-06-01,3,4111.11000,수입,러시아
3,가자미,4.50000,18500,normal,2022-06-01,3,4111.11000,수입,러시아
4,가자미,4.50000,14000,normal,2022-06-03,3,3111.11000,수입,러시아
...,...,...,...,...,...,...,...,...,...
1070137,토마토,10.00000,4000,normal,2022-06-29,7,400.00000,인천,강화
1070138,토마토,10.00000,8000,normal,2022-06-30,7,800.00000,인천,강화
1070139,토마토,5.00000,4000,normal,2022-06-30,7,800.00000,인천,강화
1070140,토마토,10.00000,7500,normal,2022-06-30,7,750.00000,강원,춘천


### prd
- 품목명 정리
- 품목 종류 정리

In [ ]:
# 대부분 [상품 종류]상품명 으로 정리되어 있는걸 알 수 있음
df2['prd'].value_counts()

[참외]참외            60100
[오이]백다다기          40985
[참외]금싸라기(참외)      31324
[참외]금싸라기          25738
[깻잎]깻잎            24122
                  ...  
[청어수입]청어수입            1
[가오리수입]가오리수입          1
[기타엽채]기타엽채(비름)        1
[피뿔고동국산]피뿔고동국산        1
[복숭아]단금도(복숭아)         1
Name: prd, Length: 1469, dtype: int64

In [ ]:
# 정리를 위해 상품명 삭제
def prd_sort(words):
  p = re.compile('[ㄱ-힣]+')
  word = re.search(p,words)
  return word.group()

df2['prd'] = df2['prd'].apply(prd_sort)

In [ ]:
# 기타 품목 삭제
df2 = df2.drop(df2[df2['prd']=='기타'].index)

In [ ]:
# 국산,수입산,깐,기타 삭제
def prd_sort2(words):
  p = re.compile('[^국산수입깐기타]+')
  word = re.search(p,str(words))
  return word.group()

df2["prd"] = df2["prd"].apply(prd_sort2)


In [ ]:
# 몇몇 품목들 정리가 필요
a = sorted(df['prd'].unique())
for i in range(len(a)):
  if i%15 != 0:
    print(a[i],end=' ')
  else:
    print(a[i],end=' ')
    print()

가리비 
가물치 가오리 가오리채 가자미 가죽나물 가지 간재미 갈치 감 감귤 감자 갑오징어 갓 강낭콩 개불 
개조개 갯가재 건고구마순 게 게지 고 고구마 고구마순 고둥 고들빼 고등어 고사리 고추 고추잎 곤달비 
곤드레나물 골뱅이 과실류 과일 과일류 과채 굴 귤 그레이프푸룻 근대 근채류 깻잎 꼬막 꼴뚜 꽁치 
꽈리고추 낙지 넙치 논우렁이 눈볼대 느 다래 다시마 닭새우 당근 대구 대구고니 대구머리 대구포 대추 
대파 도다리 돗나물 동부콩 동자개 동죽 두류 두릅 두리안 듀리안 딸 땅콩 라임 람부탄 레몬 
리치 마늘 만가닥 만감 맛 망고 망고스턴 망고스틴 매실 머위대 메 메론 명태 명태곤니 명태알 
명태포 모시조개 무 무순 무우 무화과 문어 문어바지락 미꾸라지 미나리 미더덕 민들레 민물새우 바나나 바지락 
박 밤 방울토마토 방풍 방풍나물 배 배추 백새우 백조 백합 뱀장어 버섯 버섯류 버찌 벗 
병어 보리 복분자 복숭아 봉지굴 부세 부지갱이 부추 북방대합조개 붉은고추 붕어 붕장어 브로커리 브로코리 블루베리 
비름 비트 빈스 빙어 사과 살구 삼 삼치 상추 새꼬막 새송이 새싹 새우 새우살 새조개 
샐러리 생강 생고추 세러리 셀러리 소라 속새 숙주나물 숫꽃게 시금치 시바 실파 쌈채류 쑥 쑥갓 
아귀 아보카도 아보카드 아스파라가스 아스파라거스 아욱 알 알배 앵두 양배추 양상추 양송이 양채 양채류 양파 
얼갈이 얼갈이배추 연근 연어 열무 염고등어 엽경채류 엽채 엽채류 오디 오렌지 오만둥이 오이 오징어 오징어다리 
오징어원양 오징어채 옥 완두콩 용과 우렁쉥이 우렁이 유채 임연 잉어 자두 자라 자몽 재첩 적근대 
적새우 적채 절단 조개살 죽순 쪽파 쭈꾸미 참나물 참다래 참외 참조 참죽나무순 채소 천도복숭아 청경채 
청어 체리 총각무 취나물 치커리 칠게 칼라후라워 칼리플라워 칼리후라워 케일 코라비 코코넛 콜라비 콩 콩나물 
키위 키조개 키조개날개 토마토 파 파세리 파인애플 파파야 파프리카 패류 팽이 팽이버섯 포도 포장바지락 표고 
표고버섯 풋고추 풋옥 피망 피뿔고동 피조개 학꽁치 한치

In [ ]:
### 예시
# df2[(df2['prd']=='유채') & (df2['price']==4500)]['prd'].unique()
# df2[df2['prd']=='유채']['prd'].unique()

In [ ]:
### 원래 단어 뭐였는지 검색
# org_df[org_df['품목명'].str.contains('느')]['품목명'].unique()

array(['[기타패류국산]기타패류국산'], dtype=object)

In [ ]:
### 현재 이름 검색
# df2[df2['prd']=='느']

,prd,scale,class,price,origin,eco,reg_date
1022325,유채,1kg,특(1등),4500,전남 고흥군,일반,20220629


In [ ]:
###
# JG

# 고 -> 고수로 변경
df2.loc[df2["prd"]=="고","prd"] = "고수"
# 과실류, 과일류, 과채, 채소 -> 과채류로 통합
df2.loc[df2["prd"]=="과실류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일","prd"] = "과채류"
df2.loc[df2["prd"]=="과채","prd"] = "과채류"
df2.loc[df2["prd"]=="채소","prd"] = "과채류"
# 고들빼 -> 고들빼기로 변경
df2.loc[df2["prd"]=="고들빼","prd"] = "고들빼기"
# 꼴뚜 -> 꼴두기로 변경
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
# 그레이프푸룻 -> 자몽으로 통합
df2.loc[df2["prd"]=="그레이프푸룻","prd"] = "자몽"

# 느 -> 느타리버섯으로 통합
df2.loc[df2["prd"]=="느","prd"] = "느타리버섯"

# 대구 관련 품목들 정리
df2.loc[df2["prd"]=="대구고니","prd"] = "대구"
df2.loc[df2["prd"]=="대구머리","prd"] = "대구"
df2.loc[df2["prd"]=="대구포","prd"] = "대구"
# 듀리안 -> 두리안으로 통합
df2.loc[df2["prd"]=="듀리안","prd"] = "두리안"
# 딸 -> 딸기로 변경
df2.loc[df2["prd"]=="딸","prd"] = "딸기"

# 만가닥 -> 만가닥버섯으로 변경
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
# 맛 -> 맛조개로 변경
df2.loc[df2["prd"]=="맛","prd"] = "맛조개"
# 망고스턴 -> 망고스틴으로 수정
df2.loc[df2["prd"]=="망고스턴","prd"] = "망고스틴"
# 메 -> 메기로 변경
df2.loc[df2["prd"]=="메","prd"] = "메기"
# 명태 관련 품목들 정리
df2.loc[df2["prd"]=="명태곤니","prd"] = "명태"
df2.loc[df2["prd"]=="명태알","prd"] = "명태"
df2.loc[df2["prd"]=="명태포","prd"] = "명태"
# 무우 -> 무로 통합
df2.loc[df2["prd"]=="무우","prd"] = "무"

# 방풍 -> 방풍나물로 통합
df2.loc[df2["prd"]=="방풍","prd"] = "방풍나물"
# 버섯류 -> 버섯으로 변경
df2.loc[df2["prd"]=="버섯","prd"] = "버섯"
# 벗,버찌 -> 체리로 통합
df2.loc[df2["prd"]=="벗","prd"] = "체리"
df2.loc[df2["prd"]=="버찌","prd"] = "체리"
# 브로커리, 브로코리 -> 브로콜리로 통합
df2.loc[df2["prd"]=="브로커리","prd"] = "브로콜리"
df2.loc[df2["prd"]=="브로코리","prd"] = "브로콜리"

# 삼 -> 수삼으로 변경
df2.loc[df2["prd"]=="삼","prd"] = "수삼"
# 새우살 -> 새우로 변경
df2.loc[df2["prd"]=="새우살","prd"] = "새우"
# 세러리, 셀러리 -> 샐러리로 통합
df2.loc[df2["prd"]=="세러리","prd"] = "샐러리"
df2.loc[df2["prd"]=="셀러리","prd"] = "샐러리"
# 생고추 -> 고추로 통합
df2.loc[df2["prd"]=="생고추","prd"] = "고추"

# 아보카드 -> 아보카도로 변경
df2.loc[df2["prd"]=="아보카드","prd"] = "아보카도"
# 아스파라가스 -> 아스파라거스로 변경
df2.loc[df2["prd"]=="아스파라가스","prd"] = "아스파라거스"
# 알 -> 알타리무로 변경
df2.loc[df2["prd"]=="알","prd"] = "알타리무"
# 알배 -> 알배기로 변경
df2.loc[df2["prd"]=="알배","prd"] = "알배기"
# 양채 -> 양채류로 통합
df2.loc[df2["prd"]=="양채","prd"] = "양채류"
# 얼갈이 -> 얼갈이배추로 통합
df2.loc[df2["prd"]=="얼갈이","prd"] = "얼갈이배추"
# 엽채류, 엽채 -> 엽경채류로 통합
df2.loc[df2["prd"]=="엽채류","prd"] = "엽경채류"
df2.loc[df2["prd"]=="엽채","prd"] = "엽경채류"
# 오징어 관련 품목들 정리
df2.loc[df2["prd"]=="오징어다리","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어원양","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채","prd"] = "오징어"
df2.loc[df2["prd"]=="활복오징어","prd"] = "오징어"
# 옥, 풋옥 -> 옥수수로 정리
df2.loc[df2["prd"]=="옥","prd"] = "옥수수"
df2.loc[df2["prd"]=="풋옥","prd"] = "옥수수"
# 임연 -> 임연수어로 정리
df2.loc[df2["prd"]=="임연","prd"] = "임연수어"


# 절단 -> 절단쭈꾸미 -> 쭈꾸미로 정리
df2.loc[df2["prd"]=="절단","prd"] = "쭈꾸미"
# 조개살 -> 조개로 정리
df2.loc[df2["prd"]=="조개살","prd"] = "조개"

# 참조 -> 참조기로 정리
df2.loc[df2["prd"]=="참조","prd"] = "참조기"

# 칼라후라워,칼리플라워, 칼리후라워 -> 컬리플라워로 통합
df2.loc[df2["prd"]=="칼라후라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리플라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리후라워","prd"] = "컬리플라워"
# 코라비 -> 콜라비로 통합
df2.loc[df2["prd"]=="코라비","prd"] = "콜라비"
# 키조개날개 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개날개","prd"] = "키조개"

# 팽이 -> 팽이버섯으로 통합
df2.loc[df2["prd"]=="팽이","prd"] = "팽이버섯"
# 파세리 -> 파슬리로 변경
df2.loc[df2["prd"]=="파세리","prd"] = "파슬리"
# 표고 -> 표고버섯으로 통합
df2.loc[df2["prd"]=="표고","prd"] = "표고버섯"
# 포장바지락 -> 바지락으로 변경
df2.loc[df2["prd"]=="포장바지락","prd"] = "바지락"

### 일단 여기까지 실행
# JY



In [ ]:
### 여기서 처리할거 확인, 리스트 복붙하고 고치기
a = sorted(df['prd'].unique())
for i in range(len(a)):
  if i%15 != 0:
    print(a[i],end=' ')
  else:
    print(a[i],end=' ')
    print()

가리비 
가물치 가오리 가오리채 가자미 가죽나물 가지 간재미 갈치 감 감귤 감자 갑오징어 갓 강낭콩 개불 
개조개 갯가재 건고구마순 게 게지 고구마 고구마순 고둥 고들빼기 고등어 고사리 고수 고추 고추잎 곤달비 
곤드레나물 골뱅이 과채류 굴 귤 근대 근채류 깻잎 꼬막 꼴뚜기 꽁치 꽈리고추 낙지 넙치 논우렁이 
눈볼대 느타리버섯 다래 다시마 닭새우 당근 대구 대추 대파 도다리 돗나물 동부콩 동자개 동죽 두류 
두릅 두리안 딸기 땅콩 라임 람부탄 레몬 리치 마늘 만가닥버섯 만감 맛조개 망고 망고스틴 매실 
머위대 메기 메론 명태 모시조개 무 무순 무화과 문어 문어바지락 미꾸라지 미나리 미더덕 민들레 민물새우 
바나나 바지락 박 밤 방울토마토 방풍나물 배 배추 백새우 백조 백합 뱀장어 버섯 버섯류 병어 
보리 복분자 복숭아 봉지굴 부세 부지갱이 부추 북방대합조개 붉은고추 붕어 붕장어 브로콜리 블루베리 비름 비트 
빈스 빙어 사과 살구 삼치 상추 새꼬막 새송이 새싹 새우 새조개 샐러리 생강 생고추 소라 
속새 수삼 숙주나물 숫꽃게 시금치 시바 실파 쌈채류 쑥 쑥갓 아귀 아보카도 아스파라거스 아욱 알배기 
알타리무 앵두 양배추 양상추 양송이 양채류 양파 얼갈이배추 연근 연어 열무 염고등어 엽경채류 오디 오렌지 
오만둥이 오이 오징어 옥수수 완두콩 용과 우렁쉥이 우렁이 유채 임연수어 잉어 자두 자라 자몽 재첩 
적근대 적새우 적채 조개 죽순 쪽파 쭈꾸미 참나물 참다래 참외 참조기 참죽나무순 천도복숭아 청경채 청어 
체리 총각무 취나물 치커리 칠게 컬리플라워 케일 코코넛 콜라비 콩 콩나물 키위 키조개 토마토 파 
파슬리 파인애플 파파야 파프리카 패류 팽이버섯 포도 표고버섯 풋고추 피망 피뿔고동 피조개 학꽁치 한치 함초 
해삼 해파리 향나물 호박 호박잎 홍고추 홍어 홍합 황새치 

In [ ]:
# 전체 거래량과 품목량
# 비교를 위해 분포 파악
# 거래량이 너무 적은 품목은 삭제할 필요성이 있음

prd_lst = (df['prd'].value_counts()).to_frame()
prd_lst.describe()

,prd
count,235.000000
mean,4558.012766
std,13339.881686
min,1.000000
25%,14.000000
50%,165.000000
75%,2313.500000
max,129040.000000


In [ ]:
# 50개 미만 품목은 비교가 힘드므로 삭제
# 줄어든 종류만 남기기

prd_lst = prd_lst[prd_lst['prd']>=50]
prd_lst.reset_index(inplace=True)
prd_lst = prd_lst.rename(columns = {'index':'prd','prd':'count'})

In [ ]:
df2 = df.merge(prd_lst)
df2[df2['count']>=50]['prd'].value_counts()

참외       129040
버섯        74905
오이        64719
토마토       56705
복숭아       54264
          ...  
재첩           64
개조개          64
참죽나무순        63
죽순           58
갈치           53
Name: prd, Length: 151, dtype: int64

In [ ]:
# 줄어든 것 확인했으므로 count column은 삭제
df2 = df2.drop(columns='count')

### scale

In [ ]:
# 현재 값들은 string이므로 float 형태로 변환할 필요가 있음
# 몇몇 이상치 처리도 진행
def scale_checker(scale):
  if scale.startswith('.')==True:
    return format(float('0'+scale[:-2]),'.2f')
  else:
    return format(float((scale[:-2])),'.2f')
df2['scale'] = df2['scale'].apply(scale_checker)
# df2['scale'] = format(round(df2['scale'].apply(pd.to_numeric),2),'.2f')

In [ ]:
df2['scale'] = df2['scale'].astype(float)
np.set_printoptions(precision=2, suppress=True)
df2['scale'].unique()

array([    4.5 ,     5.  ,    18.  ,     3.  ,    19.  ,     8.  ,
           8.01,     0.6 ,     1.5 ,     7.  ,     5.01,     6.  ,
           4.  ,     1.  ,     0.4 ,    10.  ,    20.  ,    15.  ,
           0.5 ,     9.6 ,    10.4 ,     9.  ,     5.5 ,     1.2 ,
           0.8 ,    12.  ,     2.  ,    13.  ,    11.  ,    17.  ,
          14.  ,    21.  ,    25.  ,    23.  ,    16.  ,     7.5 ,
           3.2 ,     3.6 ,     3.5 ,     2.6 ,     3.8 ,     0.2 ,
           2.2 ,     2.5 ,     3.4 ,     2.7 ,     2.1 ,     3.7 ,
           1.4 ,     2.4 ,     2.8 ,     2.3 ,     3.3 ,     1.6 ,
           6.5 ,     5.4 ,     8.5 ,     8.3 ,     1.7 ,     0.75,
           3.1 ,     2.9 ,     9.4 ,     5.2 ,     6.8 ,     1.8 ,
           6.6 ,   100.  ,     6.7 ,     5.7 ,     8.6 ,    11.5 ,
           9.5 ,     7.4 ,     0.1 ,     7.6 ,     6.4 ,     5.1 ,
           0.25,     0.15,     0.06,     0.05,     0.13,     0.09,
           0.9 ,     0.07,     0.7 ,     0.16,    22.  ,    24

In [ ]:
# 50kg 이상 거래된 건들 이상치라 판단하고 확인 진행
df2[df2['scale']>=50]

,prd,scale,class,price,origin,eco,reg_date
85186,고추,100.0,특(1등),27000,전남 곡성군,일반,20220624
208083,당근,200.0,특(1등),32000,경남 김해시,일반,20220627
208085,당근,200.0,특(1등),38000,경남 김해시,일반,20220627
208117,당근,200.0,특(1등),23000,경남 김해시,일반,20220627
208118,당근,200.0,특(1등),23000,경남 김해시,일반,20220627
208119,당근,200.0,특(1등),23000,경남 김해시,일반,20220627
208120,당근,200.0,특(1등),27000,경남 김해시,일반,20220627
208121,당근,200.0,특(1등),23000,경남 김해시,일반,20220627
208122,당근,200.0,특(1등),23000,경남 김해시,일반,20220627
208152,당근,200.0,특(1등),23000,경남 김해시,일반,20220627


In [ ]:
# 당근 200kg 거래의 경우 가격을 비교해보니 20kg 치 거래가 오표기 된 듯. 수정
df2.loc[(df2['prd']=='당근') & (df2['scale']==200),'scale'] = 20

In [ ]:
# 나머지는 이상치로 판단하고 모두 삭제
df2 = df2.drop(df2[df2['scale']>=50].index)

### price

In [ ]:
# 1원짜리 거래 발견, 너무 거래값이 작은 값들은 삭제
pd.options.display.float_format = '{:.5f}'.format
df2['price'].describe()

count    1070142.00000
mean       18195.36898
std        50759.90495
min            1.00000
25%         8000.00000
50%        13000.00000
75%        21000.00000
max     46315500.00000
Name: price, dtype: float64

In [ ]:
df2 = df2.drop(df2[df2['price']<500].index)

In [ ]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,가자미,4.50000,18500,수입 러시아\t,일반,20220601,3
1,가자미,4.50000,16000,수입 러시아\t,일반,20220601,3
2,가자미,4.50000,18500,수입 러시아\t,일반,20220601,3
3,가자미,4.50000,18500,수입 러시아\t,일반,20220601,3
4,가자미,4.50000,14000,수입 러시아\t,일반,20220603,3
...,...,...,...,...,...,...,...
1070137,토마토,10.00000,4000,인천 강화군,일반,20220629,7
1070138,토마토,10.00000,8000,인천 강화군,일반,20220630,7
1070139,토마토,5.00000,4000,인천 강화군,일반,20220630,7
1070140,토마토,10.00000,7500,강원 춘천시,일반,20220630,7


In [ ]:
# kg 당 가격 생성

df2['price_kg'] = round((df2['price']/df2['scale']),2)

### origin

In [ ]:
# 도,시, 국산, 수입 표기 분류 필요
df2['origin'].unique()

array(['수입 러시아\t', '전북 군산\t', '수입 미국\t', '경기 안성시', '경기 여주시', '경기 용인시',
       '경남 창녕군', '경북 성주군', '경기 이천시', '경기 하남시', '경남 밀양시', '경남 창원시 의창구',
       '경기도 하남시', '경기도 여주시 능서면', '광주 광산구', '경기도 여주시 점동면', '경남 진주시',
       '경기 여주군', '광주시 광산구', '경기 광주시', '충남 아산시', '경기 여주시 여주읍', '경북 영주시',
       '경기 고양시', '수입 기타\t', '수입 세네갈\t', '제주 제주도\t', '국내산\t', '수입 오만\t',
       '제주 서귀포시', '제주 제주시', '전남 보성군', '서울시 강서구', '충남 당진시', '충남 부여군',
       '충남 서산시', '경북 구미시', '강원 횡성군', '충남 서산시 부석면', '충남 예산군', '전라북도 부안군',
       '충남 논산시', '전북 김제시', '전북 남원시', '충북 진천군 초평면', '강원 평창군', '경남 김해시',
       '전북 부안군', '강원 홍천군', '경남 창원시 성산구', '충남 홍성군', '충북 진천군', '충남 서천군',
       '전북 고창군 신림면', '충남 태안군', '전남 곡성군', '인천시 강화군', '경기 김포시', '경기 양평군',
       '충남 보령시', '강원도 철원군', '전남 구례군', '전북 익산시', '충남 천안시', '경기도 김포시',
       '전남 고흥군', '강원 화천군', '충남 금산군', '경기 양주시', '충남 논산시 성동면', '충북 괴산군',
       '전남 해남군', '경북 예천군', '경기 평택시', '전남 무안군', '충북 청원군', '충북 충주시',
       '강원 춘천시', '전남 장흥군', '경북 칠곡군', '충북 청주시', '청주시 청원구', '강원 양구군',
       '전남 순천시'

In [ ]:
# 해당 조건에 맞는 값 추출

splits = df2['origin'].str.split(' ')
cacul = splits.apply(lambda x: pd.Series(x))
df3 = pd.merge(df2,cacul, left_index=True, right_index=True, how='inner')
df3

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,0,1,2,3
0,가자미,4.50000,18500,수입 러시아\t,일반,20220601,3,4111.11000,수입,러시아\t,NaN,NaN
1,가자미,4.50000,16000,수입 러시아\t,일반,20220601,3,3555.56000,수입,러시아\t,NaN,NaN
2,가자미,4.50000,18500,수입 러시아\t,일반,20220601,3,4111.11000,수입,러시아\t,NaN,NaN
3,가자미,4.50000,18500,수입 러시아\t,일반,20220601,3,4111.11000,수입,러시아\t,NaN,NaN
4,가자미,4.50000,14000,수입 러시아\t,일반,20220603,3,3111.11000,수입,러시아\t,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1070137,토마토,10.00000,4000,인천 강화군,일반,20220629,7,400.00000,인천,강화군,NaN,NaN
1070138,토마토,10.00000,8000,인천 강화군,일반,20220630,7,800.00000,인천,강화군,NaN,NaN
1070139,토마토,5.00000,4000,인천 강화군,일반,20220630,7,800.00000,인천,강화군,NaN,NaN
1070140,토마토,10.00000,7500,강원 춘천시,일반,20220630,7,750.00000,강원,춘천시,NaN,NaN


In [ ]:
# 세부 카테고리 정리

df4 = df3.rename(columns = {0:'state',1:'city'})

# 2,3은 필요없는 정보이므로 삭제
df4 = df4.drop(columns=[2,3])

In [ ]:
df4['state'].unique()

array(['수입', '전북', '경기', '경남', '경북', '경기도', '광주', '광주시', '충남', '제주',
       '국내산\t', '전남', '서울시', '강원', '전라북도', '충북', '인천시', '강원도', '청주시',
       '서울', '인천', '대구', '서울특별시', '부산', '충청남도', '제주도', '대구시', '대전', '전주시',
       '포클랜드\t', '칠레', '성남시', '호주', '수입산', '세종시', '타이', '태국', '베트남',
       '필리핀', '멕시코', '미국', '우즈베키스탄', '뉴질랜드', '중국', '경기고양시일산구', '충북청주시상당구',
       '전북전주시완산구', '캘리포니아', '페루', '대만', '경상남도', '부천시', '에콰도르', '콜롬비아',
       '에쿠아도르', '과테말라', '코스타리카', '기타외국', '우즈베크', '세종', '대전시', '제주시',
       '우크라이나', '부산시', '가락동', '경북/문경시', '스페인', '이스라엘', '남아프리카',
       '남아프리카공화국', '수원시', '인천광역시', '오스트레일', '전라남도'], dtype=object)

In [ ]:
# 시,도 제거
def name_sort(words):
  p = re.compile('[^시도]+')
  word = re.search(p,str(words))
  return word.group()

df4["state"] = df4["state"].apply(name_sort)

# 오타도 제거

def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["state"] = df4["state"].apply(name_sort2)

In [ ]:
df4["state"].unique()

array(['수입', '전북', '경기', '경남', '경북', '광주', '충남', '제주', '국내산', '전남', '서울',
       '강원', '전라북', '충북', '인천', '청주', '대구', '서울특별', '부산', '충청남', '대전',
       '전주', '포클랜드', '칠레', '성남', '호주', '수입산', '세종', '타이', '태국', '베트남',
       '필리핀', '멕', '미국', '우즈베키스탄', '뉴질랜드', '중국', '경기고양', '충북청주', '전북전주',
       '캘리포니아', '페루', '대만', '경상남', '부천', '에콰', '콜롬비아', '에쿠아', '과테말라',
       '코스타리카', '기타외국', '우즈베크', '우크라이나', '가락동', '경북/문경', '스페인', '이스라엘',
       '남아프리카', '남아프리카공화국', '수원', '인천광역', '오스트레일', '전라남'], dtype=object)

In [ ]:
# state
# 특이케이스 제거

# 국내
df4.loc[df4["state"]=="국내산","city"] = "미상"
df4.loc[df4["state"]=="국내산","state"] = "국산"

## 도단위
df4.loc[df4["state"]=="충청북","state"] = "충북"
df4.loc[df4["state"]=="충청남","state"] = "충남"
df4.loc[df4["state"]=="전라북","state"] = "전북"
df4.loc[df4["state"]=="전라남","state"] = "전남"
df4.loc[df4["state"]=="경상북","state"] = "경북"
df4.loc[df4["state"]=="경상남","state"] = "경남"

## 시단위
### 광역시

df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="대구","city"] = "대구"
df4.loc[df4["state"]=="서울특별","city"] = "서울"
df4.loc[df4["state"]=="서울특별","state"] = "서울"
df4.loc[df4["state"]=="인천광역","city"] = "인천"
df4.loc[df4["state"]=="인천광역","state"] = "인천"
df4.loc[df4["state"]=="부산","city"] = "부산"
df4.loc[df4["state"]=="가락동","city"] = "서울"
df4.loc[df4["state"]=="가락동","state"] = "서울"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="제주","city"] = "제주"
df4.loc[df4["state"]=="세종","city"] = "세종"

### 일반시
df4.loc[df4["state"]=="성남","city"] = "성남"
df4.loc[df4["state"]=="성남","state"] = "경기"
df4.loc[df4["state"]=="전주","city"] = "전주"
df4.loc[df4["state"]=="전주","state"] = "전북"
df4.loc[df4["state"]=="청주","city"] = "청주"
df4.loc[df4["state"]=="청주","state"] = "충북"
df4.loc[df4["state"]=="수원","city"] = "수원"
df4.loc[df4["state"]=="수원","state"] = "경기"
df4.loc[df4["state"]=="부천","city"] = "부천"
df4.loc[df4["state"]=="부천","state"] = "경기"
df4.loc[df4["state"]=="경기고양","city"] = "고양"
df4.loc[df4["state"]=="경기고양","state"] = "경기"
df4.loc[df4["state"]=="충북청주","city"] = "청주"
df4.loc[df4["state"]=="충북청주","state"] = "충북"
df4.loc[df4["state"]=="전북전주","city"] = "전주"
df4.loc[df4["state"]=="전북전주","state"] = "전북"
df4.loc[df4["state"]=="경북/문경","city"] = "문경"
df4.loc[df4["state"]=="경북/문경","state"] = "경북"

# 외국
## 아시아
df4.loc[df4["state"]=="칠레","city"] = "칠레"
df4.loc[df4["state"]=="칠레","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="우즈베크","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베크","state"] = "수입"
df4.loc[df4["state"]=="이스라엘","city"] = "이스라엘"
df4.loc[df4["state"]=="이스라엘","state"] = "수입"
df4.loc[df4["state"]=="터키","city"] = "터키"
df4.loc[df4["state"]=="터키","state"] = "수입"
df4.loc[df4["state"]=="중국","city"] = "중국"
df4.loc[df4["state"]=="중국","state"] = "수입"
df4.loc[df4["state"]=="타이","city"] = "태국"
df4.loc[df4["state"]=="타이","state"] = "수입"
df4.loc[df4["state"]=="태국","city"] = "태국"
df4.loc[df4["state"]=="태국","state"] = "수입"
df4.loc[df4["state"]=="필리핀","city"] = "필리핀"
df4.loc[df4["state"]=="필리핀","state"] = "수입"
df4.loc[df4["state"]=="우즈베키스탄","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베키스탄","state"] = "수입"
df4.loc[df4["state"]=="대만","city"] = "대만"
df4.loc[df4["state"]=="대만","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"

## 아메리카

df4.loc[df4["state"]=="캘리포니아","city"] = "미국"
df4.loc[df4["state"]=="캘리포니아","state"] = "수입"
df4.loc[df4["state"]=="포클랜드","city"] = "영국"
df4.loc[df4["state"]=="포클랜드","state"] = "수입"
df4.loc[df4["state"]=="에콰","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰","state"] = "수입"
df4.loc[df4["state"]=="콜롬비아","city"] = "콜롬비아"
df4.loc[df4["state"]=="콜롬비아","state"] = "수입"
df4.loc[df4["state"]=="에쿠아","city"] = "에콰도르"
df4.loc[df4["state"]=="에쿠아","state"] = "수입"
df4.loc[df4["state"]=="과테말라","city"] = "과테말라"
df4.loc[df4["state"]=="과테말라","state"] = "수입"
df4.loc[df4["state"]=="코스타리카","city"] = "코스타리카"
df4.loc[df4["state"]=="코스타리카","state"] = "수입"
df4.loc[df4["state"]=="멕","city"] = "멕시코"
df4.loc[df4["state"]=="멕","state"] = "수입"
df4.loc[df4["state"]=="페루","city"] = "페루"
df4.loc[df4["state"]=="페루","state"] = "수입"
df4.loc[df4["state"]=="미국","city"] = "미국"
df4.loc[df4["state"]=="미국","state"] = "수입"

## 유럽
df4.loc[df4["state"]=="스페인","city"] = "스페인"
df4.loc[df4["state"]=="스페인","state"] = "수입"
df4.loc[df4["state"]=="우크라이나","city"] = "우크라이나"
df4.loc[df4["state"]=="우크라이나","state"] = "수입"

## 아프리카
df4.loc[df4["state"]=="남아프리카","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카","state"] = "수입"
df4.loc[df4["state"]=="남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카공화국","state"] = "수입"

## 오세아니아
df4.loc[df4["state"]=="뉴질랜드","city"] = "뉴질랜드"
df4.loc[df4["state"]=="뉴질랜드","state"] = "수입"
df4.loc[df4["state"]=="오스트레일","city"] = "호주"
df4.loc[df4["state"]=="오스트레일","state"] = "수입"
df4.loc[df4["state"]=="호주","city"] = "호주"
df4.loc[df4["state"]=="호주","state"] = "수입"

# 기타
df4.loc[df4["state"]=="수입산","state"] = "수입"
df4.loc[df4["state"]=="기타외국","city"] = "미상"
df4.loc[df4["state"]=="기타외국","state"] = "수입"


In [ ]:
df4[df4['state']=='nan']

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,state,city


In [ ]:
df4['state'].unique()

array(['수입', '전북', '경기', '경남', '경북', '광주', '충남', '제주', '국산', '전남', '서울',
       '강원', '충북', '인천', '대구', '부산', '대전', '세종'], dtype=object)

In [ ]:
df4['city'].unique()

array(['러시아\t', '군산\t', '미국\t', '안성시', '여주시', '용인시', '창녕군', '성주군', '이천시',
       '하남시', '밀양시', '창원시', '광산구', '진주시', '여주군', '광주시', '아산시', '영주시',
       '고양시', '기타\t', '세네갈\t', '제주', '미상', '오만\t', '보성군', '강서구', '당진시',
       '부여군', '서산시', '구미시', '횡성군', '예산군', '부안군', '논산시', '김제시', '남원시',
       '진천군', '평창군', '김해시', '홍천군', '홍성군', '서천군', '고창군', '태안군', '곡성군',
       '강화군', '김포시', '양평군', '보령시', '철원군', '구례군', '익산시', '천안시', '고흥군',
       '화천군', '금산군', '양주시', '괴산군', '해남군', '예천군', '평택시', '무안군', '청원군',
       '충주시', '춘천시', '장흥군', '칠곡군', '청주시', '청주', '양구군', '순천시', '영광군',
       '공주시', '강릉시', '원주시', '사천시', '김천시', nan, '광명시', '상주시', '음성군', '옥천군',
       '포천시', '계양구', '안동시', '대구', '여수\t', '장항(서천)\t', '베트남\t', '서산\t',
       '통영\t', '영암군', '완주군', '삼천포\t', '인천\t', '노르웨이\t', '네덜란드\t', '남구',
       '고성군', '홍천', '전주시', '진도군', '서울', '중국\t', '남해\t', '순천(광양)\t',
       '부안\t', '고창\t', '대천(보령)\t', '마산(고성,진해)', '부산', '장수군', '남양주시',
       '완도군', '의성군', '청양군', '파주시', '광양시', '순창군', '영천시', '나주시', '화성(평택)\t',
     

In [ ]:
# city
# 우선 결측치 제거
df4.loc[df4["city"].isnull(),"city"] = "미상"

In [ ]:
# 이후 시,군 등 제거
def name_sort(words):
  p = re.compile('[^시군]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort)

In [ ]:
# \t 도 제거
def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort2)

In [ ]:
df4['city'].unique()

array(['러', '산', '미국', '안성', '여주', '용인', '창녕', '성주', '이천', '하남', '밀양',
       '창원', '광산구', '진주', '광주', '아산', '영주', '고양', '기타', '세네갈', '제주', '미상',
       '오만', '보성', '강서구', '당진', '부여', '서산', '구미', '횡성', '예산', '부안', '논산',
       '김제', '남원', '진천', '평창', '김해', '홍천', '홍성', '서천', '고창', '태안', '곡성',
       '강화', '김포', '양평', '보령', '철원', '구례', '익산', '천안', '고흥', '화천', '금산',
       '양주', '괴산', '해남', '예천', '평택', '무안', '청원', '충주', '춘천', '장흥', '칠곡',
       '청주', '양구', '순천', '영광', '공주', '강릉', '원주', '사천', '김천', '광명', '상주',
       '음성', '옥천', '포천', '계양구', '안동', '대구', '여수', '장항(서천)', '베트남', '통영',
       '영암', '완주', '삼천포', '인천', '노르웨이', '네덜란드', '남구', '고성', '전주', '진도',
       '서울', '중국', '남해', '순천(광양)', '대천(보령)', '마산(고성,진해)', '부산', '장수',
       '남양주', '완도', '의성', '청양', '파주', '광양', '순창', '영천', '나주', '화성(평택)',
       '구리', '은평구', '담양', '경주', '화성', '화순', '정읍', '의정부', '문경', '영덕', '영월',
       '영양', '서구', '강동구', '연천', '제천', '봉화', '인제', '삼척', '양양', '경산', '하동',
       '영동', '함안', '청송', '산청', '함평', '함양', '에콰도르', '

In [ ]:
# 나머지 케이스 정리
df4.loc[df4["city"]=="러","city"] = "러시아"
df4.loc[df4["city"]=="말레이지아","city"] = "말레이시아"
df4.loc[df4["city"]=="광산구","city"] = "광주"
df4.loc[df4["city"]=="산","city"] = "군산"
df4.loc[df4["city"]=="장항(서천)","city"] = "서천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="남구"),"city"] = "인천"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="남구"),"city"] = "광주"
df4.loc[df4["city"]=="순천(광양)","city"] = "순천"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="마산(고성,진해)","city"] = "마산"
df4.loc[df4["city"]=="강서구","city"] = "서울"
df4.loc[(df4["state"]=="부산")&(df4["city"]=="북구"),"city"] = "부산"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="북구"),"city"] = "광주"
df4.loc[df4["city"]=="화성(평택)","city"] = "화성"
df4.loc[df4["city"]=="은평구","city"] = "서울"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="서구"),"city"] = "인천"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="서구"),"city"] = "광주"
df4.loc[df4["city"]=="강동구","city"] = "서울"
df4.loc[df4["city"]=="북제주","city"] = "제주"
df4.loc[df4["city"]=="마포구","city"] = "서울"
df4.loc[df4["city"]=="기타","city"] = "미상"
df4.loc[df4["city"]=="위","city"] = "군위"
df4.loc[df4["city"]=="고흥(나로도)","city"] = "고흥"
df4.loc[df4["city"]=="오포","city"] = "부산"
df4.loc[df4["city"]=="중구","city"] = "서울"
df4.loc[df4["city"]=="송파구","city"] = "서울"
df4.loc[df4["city"]=="계양구","city"] = "인천"
df4.loc[df4["city"]=="부산진","city"] = "부산"
df4.loc[df4["city"]=="도봉구","city"] = "서울"
df4.loc[df4["city"]=="멕","city"] = "멕시코"
df4.loc[df4["city"]=="중랑구","city"] = "서울"
df4.loc[df4["city"]=="흥","city"] = "시흥"
df4.loc[df4["city"]=="양천구","city"] = "서울"
df4.loc[df4["city"]=="동구","city"] = "광주"
df4.loc[df4["city"]=="성동구","city"] = "서울"
df4.loc[df4["city"]=="강남구","city"] = "서울"
df4.loc[df4["city"]=="부평구","city"] = "인천"
df4.loc[df4["city"]=="남아공산","city"] = "남아공"
df4.loc[df4["city"]=="부산진구","city"] = "부산"
df4.loc[df4["city"]=="남동구","city"] = "인천"
df4.loc[df4["city"]=="구로구","city"] = "서울"
df4.loc[df4["city"]=="미추홀구","city"] = "인천"
df4.loc[df4["city"]=="남제주","city"] = "제주"
df4.loc[df4["city"]=="서초구","city"] = "서울"
df4.loc[df4["city"]=="미추홀","city"] = "인천"
df4.loc[df4["city"]=="사상구","city"] = "부산"
df4.loc[df4["city"]=="영등포","city"] = "서울"
df4.loc[df4["city"]=="타이","city"] = "태국"
df4.loc[df4["city"]=="동대문구","city"] = "서울"
df4.loc[df4["city"]=="영등포구","city"] = "서울"

In [ ]:
df4['city'].unique()

array(['러시아', '군산', '미국', '안성', '여주', '용인', '창녕', '성주', '이천', '하남', '밀양',
       '창원', '광주', '진주', '아산', '영주', '고양', '미상', '세네갈', '제주', '오만', '보성',
       '서울', '당진', '부여', '서산', '구미', '횡성', '예산', '부안', '논산', '김제', '남원',
       '진천', '평창', '김해', '홍천', '홍성', '서천', '고창', '태안', '곡성', '강화', '김포',
       '양평', '보령', '철원', '구례', '익산', '천안', '고흥', '화천', '금산', '양주', '괴산',
       '해남', '예천', '평택', '무안', '청원', '충주', '춘천', '장흥', '칠곡', '청주', '양구',
       '순천', '영광', '공주', '강릉', '원주', '사천', '김천', '광명', '상주', '음성', '옥천',
       '포천', '인천', '안동', '대구', '여수', '베트남', '통영', '영암', '완주', '노르웨이',
       '네덜란드', '고성', '전주', '진도', '중국', '남해', '마산', '부산', '장수', '남양주',
       '완도', '의성', '청양', '파주', '광양', '순창', '영천', '나주', '화성', '구리', '담양',
       '경주', '화순', '정읍', '의정부', '문경', '영덕', '영월', '영양', '연천', '제천', '봉화',
       '인제', '삼척', '양양', '경산', '하동', '영동', '함안', '청송', '산청', '함평', '함양',
       '에콰도르', '말레이시아', '페루', '아르헨티나', '필리핀', '캐나다', '태국', '신안', '고령',
       '합천', '청도', '대전', '임실', '진안', '칠레', '영국', '의령', '

In [ ]:
# 처리 완료하였으니 df4는 백업으로 두고 df5 활용
df5 = df4.drop(columns='origin')

# 전처리 결과

# 파일 저장

In [ ]:
df5.to_csv('202206_proceed.csv')